In [193]:
from bs4 import BeautifulSoup
import requests

In [194]:
# URL = "https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=Nuggets&BeginDate=2022-01-01&EndDate=2022-04-06&PlayerMovementChkBx=yes&ILChkBx=yes&Submit=Search"
# page = requests.get(URL)

In [195]:
# soup = BeautifulSoup(page.content, "html.parser")
# table = soup.find("table", class_="datatable")
# headers = [header.text[1:] for header in table.find('tr',class_="DraftTableLabel").findAll('td')]
# results = [{headers[i]: cell.text for i, cell in enumerate(row.findAll('td'))}
#            for row in table.findAll('tr')[1:]]
# print(headers)
# print(results)

In [196]:
# for i, item in enumerate(results):
#     print(item['Acquired'][3:])

In [197]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import playergamelogs
import time

In [198]:
sleeptime = .7
headers = ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID']
rosters = {}
player_logs= {}
for team in teams.get_teams():
    team_player_logs = playergamelogs.PlayerGameLogs(season_nullable="2021-22", team_id_nullable=team["id"])
    time.sleep(sleeptime)
    player_logs[team['city'] + " " + team['nickname']] = [(player[2], player[8][:10], player[6]) for player in team_player_logs.get_dict()["resultSets"][0]['rowSet']]
    roster = [player[3] for player in commonteamroster.CommonTeamRoster(team["id"], season="2021-22").common_team_roster.get_dict()['data']]
    time.sleep(sleeptime)
    rosters[team['city'] + " " + team['nickname']]= roster
print(rosters)

{'Atlanta Hawks': ['Delon Wright', 'Jalen Johnson', 'Sharife Cooper', 'Kevin Huerter', 'Skylar Mays', 'Kevin Knox II', 'Lou Williams', 'Timothe Luwawu-Cabarrot', 'Danilo Gallinari', 'Gorgui Dieng', 'Trae Young', "De'Andre Hunter", 'Bogdan Bogdanovic', 'Clint Capela', 'Onyeka Okongwu', 'John Collins'], 'Boston Celtics': ['Matt Ryan', 'Jayson Tatum', 'Jaylen Brown', 'Malik Fitts', 'Derrick White', 'Payton Pritchard', 'Grant Williams', 'Nik Stauskas', 'Aaron Nesmith', 'Daniel Theis', 'Sam Hauser', 'Marcus Smart', 'Luke Kornet', 'Al Horford', 'Robert Williams III', 'Brodric Thomas'], 'Cleveland Cavaliers': ['Kevin Love', 'Brandon Goodwin', 'Rajon Rondo', 'Collin Sexton', 'Caris LeVert', 'Evan Mobley', 'RJ Nembhard Jr.', 'Moses Brown', 'Lamar Stevens', 'Dylan Windler', 'Darius Garland', 'Cedi Osman', 'Ed Davis', 'Lauri Markkanen', 'Jarrett Allen', 'Dean Wade', 'Isaac Okoro'], 'New Orleans Pelicans': ['Zion Williamson', 'Jared Harper', 'CJ McCollum', "Devonte' Graham", 'Herbert Jones', 'Naji

In [199]:
rosters["LA Clippers"] = rosters["Los Angeles Clippers"]
player_logs["LA Clippers"] = player_logs["Los Angeles Clippers"]

In [200]:
URL_ESPN = "https://www.espn.com/nba/injuries"
espn = requests.get(URL_ESPN)

In [201]:
soup = BeautifulSoup(espn.content, "html.parser")
tables = [table for table in soup.find_all("div", {"class": "Table__league-injuries"})]
teams_injured = [{"team":table.find("span", {"class":"injuries__teamName"}).text} for table in tables]

for i, table  in enumerate(tables):
    team = teams_injured[i]
    teams_injured[i]["injury_list"] = [{"name":item.find("a").text, "date":item.find("td",{"class":"col-date"}).text, "status":item.find("td",{"class":"col-stat"}).text} for item in table.find_all("tr", {"data-idx":True})]

In [202]:
from datetime import datetime

In [212]:

for team in teams_injured:
    team_logs = [log for log in player_logs[team["team"]] if log[2] == team["team"]]
    team_logs.sort(key=lambda date: datetime.strptime(date[1], "%Y-%m-%d"), reverse=True)
    last_team_game_played = team_logs[0][1]
    print(team["team"], last_team_game_played)
    for player in rosters[team["team"]]:
        played_with_team = [log[1] for log in team_logs if log[0] == player]
        # print([log[1] for log in team_logs])
        played_with_team.sort(key=lambda date: datetime.strptime(date, "%Y-%m-%d"), reverse=True)
        # print(played_with_team)
        if len(played_with_team) > 0:
            last_game_played_with_team = played_with_team[0]
            # print(player, last_game_played_with_team)
        else:
            last_game_played_with_team = "Never"
        if player in [player["name"] for player in team["injury_list"]]:
            if last_game_played_with_team == last_team_game_played:
                # push new injury event object with start date as today (note: handle OUT vs day-to-day)
                print(player, " Start Injury")
            else:
                #continued injury, (update games missed on object?)
                print(player, " Injured since ", last_game_played_with_team)
        # healthy
        else:
            # determine if a player has returned from injury
            if last_game_played_with_team != last_team_game_played:
                #update injury event object with end date
                print(player, " Returning from injury since", last_game_played_with_team)
            
    print("-----")

Atlanta Hawks 2022-04-06
Lou Williams  Injured since  2022-04-05
Timothe Luwawu-Cabarrot  Returning from injury since 2022-04-05
John Collins  Injured since  2022-03-11
-----
Boston Celtics 2022-04-06
Matt Ryan  Returning from injury since Never
Nik Stauskas  Injured since  2022-03-23
Robert Williams III  Injured since  2022-03-27
-----
Brooklyn Nets 2022-04-06
Blake Griffin  Returning from injury since 2022-04-02
David Duke Jr.  Returning from injury since 2022-02-28
Goran Dragic  Injured since  2022-03-31
Ben Simmons  Injured since  Never
Joe Harris  Injured since  2021-11-14
James Johnson  Injured since  2022-04-02
Day'Ron Sharpe  Returning from injury since 2022-02-28
-----
Charlotte Hornets 2022-04-05
Montrezl Harrell  Returning from injury since 2022-04-02
Scottie Lewis  Returning from injury since 2021-12-15
Gordon Hayward  Injured since  2022-04-02
Kai Jones  Returning from injury since 2022-03-02
Arnoldas Kulboka  Returning from injury since 2021-12-10
-----
Chicago Bulls 2022